In [4]:
# import the necessary packages
import tkinter as FDAS
import tkinter.ttk as tk
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2
import os
import pandas as pd
import numpy as np
import json
import requests
import geocoder

from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from numpy import load
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from tkinter import StringVar
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from csv import writer 
from datetime import datetime
from functools import partial

#Various Models and Detectors
liveliness_detector = dlib.get_frontal_face_detector()
liveliness_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
face_detector = MTCNN()

from joblib import dump, load
classifier = load('SVN_Classifier.joblib')

model = load_model('facenet_keras.h5')
from numpy import load
data = load('FDS_Dataset_Face-Embeddings.npz')

#Global Variables
speed = 0
desired_speed = 0
lastError = 0
cumError = 0
flag = 0
Driver = "Name"
Start_Time = "Time"
Start_Location = ["20° 2' 21.9192'' N","75° 29' 28.6368'' E"]
End_Time = "Time"
End_Location = ["20° 2' 21.9192'' N","75° 29' 28.6368'' E"]
New_Driver = "Name"
Auth_Flag = False

#Load list of Authorised Personnel
allowed_names =  pd.read_csv("Authorised_List.csv",sep=';')
allowed_names = allowed_names.values
allowed_names = allowed_names.ravel()

#Callback function when Ignition ON Button is pressed
def IgnitionOn():
    IGButton_On["state"] = "disabled"
    IGButton_Off["state"] = "active"
    FDAS_Button_Auth["state"] = "active"
    Message_Box_Value["state"] = "active"
    FDAS_Button_New["state"] = "active"
    Message_Box_Value['text'] = "Ignition Switch ON"    

#Callback function when Ignition OFF Button is pressed
def IgnitionOff():
    global End_Time, End_Location, Auth_Flag
    
    IGButton_On["state"] = "active"
    IGButton_Off["state"] = "disabled"
    Speed_Value["state"] = "disabled"
    Speed_Unit["state"] = "disabled"
    FDAS_Button_Auth["state"] = "disabled"
    FDAS_Button_New["state"] = "disabled"
    Accelerator_Value["state"] = "disabled"
    Message_Box_Value["state"] = "disabled"
    Message_Box_Value['text'] = "Ignition Switch OFF...\nLog Database Updated..."
    
    if Auth_Flag == True:
        End_Time =  Get_Date_Time()
        End_Location = Get_Current_Location()
        Update_Log()
        message = End_Time+": Trip ended by "+Driver[0]+" at Location: "+str(End_Location[0])+str(End_Location[1])
        Send_Push_Message(message)
        Auth_Flag = False
    
#Function to send Push Messages to the Fleet Manager
def Send_Push_Message(message):
    token = "o.XHW6XRqfYs1ngEqNPq5HABuUAFkxDZud"
    headers = {"content-type": "application/json", "Authorization": 'Bearer '+token}
    url = 'https://api.pushbullet.com/v2/pushes'
    data_send = {"type": "note", "title": "FDAS Update:-", "body": message}
    push = requests.post(url, headers=headers, data=json.dumps(data_send), verify=False)
    
#Function to get Current Date and Time
def Get_Date_Time():
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    return dt_string

#Function to get Current Location of the Vehicle GPS Module API Callback (just a demo code)
def Get_Current_Location():
    location = geocoder.ip('me')
    address = [location.address, location.latlng]
    return address
    
#Function to Update System Log
def Update_Log():
    List=[Driver, Start_Time, Start_Location[0], Start_Location[1], End_Time, End_Location[0], End_Location[1]] 
  
    with open('Log.csv', 'a') as f_object:  
        writer_object = writer(f_object) 
        writer_object.writerow(List) 
        f_object.close() 
    
#Function to grant access to authorised users
def Access_Granted():
    global Start_Time, Start_Location, Driver, Auth_Flag
    
    Speed_Value["state"] = "active"
    Speed_Unit["state"] = "active"
    Accelerator_Value["state"] = "active"
    FDAS_Button_New["state"] = "active"
    
    Start_Time =  Get_Date_Time()
    Start_Location = Get_Current_Location()
    
    message = Start_Time+": Trip started by "+Driver[0]+" from Location: "+str(Start_Location[0])+str(Start_Location[1])
    Send_Push_Message(message)
    Auth_Flag = True
    FDAS_Button_Auth["state"] = "disabled"
    
#Function to update the Accelerator variabel from GUI
def Update_Accelerator_Label(set_speed):
    global desired_speed
    desired_speed = set_speed

#Function to monitor and detect driver drowsiness
def Drowsiness_Detection():
    EYE_AR_THRESH = 0.25
    EYE_AR_CONSEC_FRAMES = 150
    COUNTER = 0
    flag = 0
    
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    
    vs = VideoStream(src=0).start()
    fileStream = False
    time.sleep(1.0)

    while True:
        if fileStream and not vs.more():
            break
            
        frame = vs.read()
        frame = imutils.resize(frame, width=450)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = liveliness_detector(gray, 0)

        for rect in rects:
            shape = liveliness_predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            (x, y, w, h) = face_utils.rect_to_bb(rect)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            for (a, b) in shape:
                cv2.circle(frame, (a, b), 1, (0, 0, 255), -1)

            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
            
            Probability = int(COUNTER * (100/150))
            if Probability > 100:
                Probability = 100

            if ear < EYE_AR_THRESH:
                COUNTER += 1
            else:
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    flag = 1
                COUNTER = 0

            cv2.putText(frame, "Drowsiness Detection ON", (2, 15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            cv2.putText(frame, "Eye_Ratio: {:.2f}".format(ear), (290, 15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            
            if(COUNTER == 0):
                cv2.putText(frame, "Eyes Open", (x, y-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                
            else:
                cv2.putText(frame, "Eyes Closed", (x, y-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                cv2.putText(frame, "{}%".format(Probability), (x+w, y-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord("q"):
            Message_Box_Value["text"] = "Drowsiness Detection aborted..."
            break
            
        if flag == 1:
            global desired_speed
            desired_speed = 0
            Message_Box_Value["text"] = "Drowsiness Detected...\nStopping Vehicle for Passenger Safety!"
            Accelerator_Value.set(0)
            
            Time =  Get_Date_Time()
            location = Get_Current_Location()
            message = Time+": Vehicle stopped due to Drowsiness Detection for "+Driver[0]+" at Location: "+str(location[0])+str(location[1])
            Send_Push_Message(message)
        
            break
            
    cv2.destroyAllWindows()
    vs.stop()  
    
#Function to simulate the linear motion of a car    
def SimulateCar():
    global speed, desired_speed, lastError, cumError, flag
    error = 0
    rateError = 0
    outputPWM = 0
    kp = 8
    ki = 0.8
    kd = 0.4
    force = 0
    friction = 0
    drag = 0
    mass = 1500
    acceleration = 0

    error = int(desired_speed) - speed
    cumError += error * 0.8
    rateOfError = (error - lastError) / 0.8
    outputPWM = (kp*error) + (ki*cumError) + (kd*rateError)
    lastError = error

    friction = 0.7 * mass * 9.8
    drag = (0.3*1.225*(speed*speed)*4)/2
    force = (outputPWM*100)-friction-drag
    acceleration = force/mass
    speed += acceleration*0.8
    
    speed = int(speed)
    
    if(speed<0):
        speed = 0
    
    if (speed>0):
        if (flag == 1):
            flag = 0
            Drowsiness_Detection()

        if (speed == int(desired_speed)):
            flag = 1
    
    Speed_Value['text'] = "{}".format(speed)
    Parent_Frame.after(500, SimulateCar)

#Function to Detect and Extract face ROI using the MTCNN model
def extract_face(image, required_size=(160, 160)):
    pixels = asarray(image)
    results = face_detector.detect_faces(pixels)
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

#Function to Load images and extract faces for all images in a directory
def load_faces(directory):
    faces = list()
    for filename in listdir(directory):
        path = directory + '/' + filename
        image = Image.open(path)
        face = extract_face(image)
        faces.append(face)
    return faces

#Function to load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = list(), list()
    for subdir in listdir(directory):
        path = directory + '/' + subdir + '/'
        faces = load_faces(path)
        labels = [subdir for _ in range(len(faces))]
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

#Function to extract and predict face embeddings using Keras CNN
def get_embedding(face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]

#Function for Facial Recognotion after Liveliness Detection is successful
def Face_Recognition(face):
    global Driver
    flag = 0
    trainy = data['arr_1']
    face_data = extract_face(face)
    face_data = get_embedding(face_data)
    result = expand_dims(face_data, axis=0)
    result = classifier.predict(result)
    out_encoder = LabelEncoder()
    out_encoder.fit(trainy)
    Driver = out_encoder.inverse_transform(result)    
    for i in allowed_names:
        if i == Driver[0]:
            Message_Box_Value["text"] = "Liveliness Detection Successful...\nFace Recognition Successful...\nAccess Granted...\nWelcome {}!".format(Driver[0])
            flag = 1
            Access_Granted()
            break
    
    if flag == 0:
        Message_Box_Value["text"] = "Liveliness Detection Successful...\nFace Recognition Un-Successful...\nUnknown Face Detected...Access Denied...!"
    

#Function to Compute Eye_Aspect_Ratio
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

#Function for Liveliness Detection at the start of Authentication Process
def Liveliness_Detection():
    EYE_AR_THRESH = 0.28
    EYE_AR_CONSEC_FRAMES = 3
    COUNTER = 0
    TOTAL = 0
    TIME_COUNTER = 0

    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    
    vs = VideoStream(src=0).start()
    fileStream = False
    time.sleep(1.0)

    while True:
        if fileStream and not vs.more():
            break
        
        TIME_COUNTER += 1  
        
        frame = vs.read()
        frame = imutils.resize(frame, width=450)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = liveliness_detector(gray, 0)

        for rect in rects:
            shape = liveliness_predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            (x, y, w, h) = face_utils.rect_to_bb(rect)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            for (a, b) in shape:
                cv2.circle(frame, (a, b), 1, (0, 0, 255), -1)

            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

            if ear < EYE_AR_THRESH:
                COUNTER += 1
            else:
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    TOTAL += 1
                COUNTER = 0
            
            Probability = TOTAL * 50
            if Probability > 100:
                Probability = 100

            cv2.putText(frame, "Face Detected", (2, 15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            cv2.putText(frame, "Eye_Ratio: {:.2f}".format(ear), (290, 15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            cv2.putText(frame, "{}%".format(Probability), (x, y-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord("q"):
            Message_Box_Value["text"] = "Liveliness Detection aborted..."
            break
            
        if TOTAL >= 2:
            Face_Recognition(frame)
            break
            
        if TIME_COUNTER > 500:
            Message_Box_Value["text"] = "Liveliness Detection Failed...\nNo Live Face Detected...\nPlease try again!"
            break
            
    cv2.destroyAllWindows()
    vs.stop()    
    
#Function to Update Database and Re-Train Model
def New_Entry_Update():
    global New_Driver, classifier, data
    name = [New_Driver]
    with open('Authorised_List.csv', 'a') as f_object:  
        writer_object = writer(f_object) 
        writer_object.writerow(name) 
        f_object.close()
    
    directory = New_Driver
    parent_dir = "C:/Users/qv445/OneDrive - Cummins/MTech Data/Disseration/Final/App/New Entries"
    final_dir = os.path.join(parent_dir, directory)
    os.mkdir(final_dir) 
    videoCaptureObject = cv2.VideoCapture(0)
    counter = 1
    frame_counter = 1
    while True:
        ret,frame = videoCaptureObject.read()
        cv2.imshow('Capturing Video for Training Model: Please look into the Camera',frame)
        if frame_counter > 150:
            if (frame_counter % 5) == 0:
                temp = str(counter) + '.jpg'
                path = os.path.join(final_dir, temp)
                cv2.imwrite(path,frame)
                counter = counter + 1
        frame_counter = frame_counter + 1
        if frame_counter > 200:
            break
        if(cv2.waitKey(1) & 0xFF == ord('q')):
            break
    videoCaptureObject.release()
    cv2.destroyAllWindows()
    
    Xtrain, Ytrain = load_dataset('C:/Users/qv445/OneDrive - Cummins/MTech Data/Disseration/Final/App/New Entries')
    trainX, testX, trainy, testy = train_test_split(Xtrain, Ytrain, random_state=1)

    newTrainX = list()
    for face_pixels in trainX:
        embedding = get_embedding(face_pixels)
        newTrainX.append(embedding)
    newTrainX = asarray(newTrainX)
    newTestX = list()
    for face_pixels in testX:
        embedding = get_embedding(face_pixels)
        newTestX.append(embedding)
    newTestX = asarray(newTestX)
    
    data_old = load('FDS_Dataset_Face-Embeddings.npz')
    trainX = np.append(data_old['arr_0'], newTrainX, 0)
    trainy = np.append(data_old['arr_1'], trainy, 0)
    testX = np.append(data_old['arr_2'], newTestX, 0)
    testy = np.append(data_old['arr_3'], testy, 0)
    savez_compressed('FDS_Dataset_Face-Embeddings.npz', trainX, trainy, testX, testy)
    
    in_encoder = Normalizer(norm='l2')
    trainX = in_encoder.transform(trainX)
    testX = in_encoder.transform(testX)
    out_encoder = LabelEncoder()
    out_encoder.fit(trainy)
    trainy = out_encoder.transform(trainy)
    testy = out_encoder.transform(testy)
    classifier = SVC(kernel='linear', probability=True)
    classifier.fit(trainX, trainy)
    yhat_train = classifier.predict(trainX)
    yhat_test = classifier.predict(testX)
    score_train = accuracy_score(trainy, yhat_train)
    score_test = accuracy_score(testy, yhat_test)
    
    FDAS_Button_Auth["state"] = "disabled"
    FDAS_Button_New["state"] = "disabled"
    Message_Box_Value["state"] = "disabled"
    
    dump(classifier, 'SVN_Classifier.joblib') 
    
    Message_Box_Value["text"] = "New Driver Entry Created...\nModel Re-Trained - Accuracy:{:.2f}%\nRestart Application to take effect!".format(score_test*100)    
    
#Callback function to validate contents of New Entry Form
def New_Entry_Validate(New_Driver_Name, Admin_Password):
    global New_Driver
    password = Admin_Password.get()
    if (password=="12345"):
        New_Driver = New_Driver_Name.get()
        New_Entry_Name_Label["state"] = "disabled"
        New_Entry_Name_Value["state"] = "disabled"
        New_Entry_Password_Label["state"] = "disabled"
        New_Entry_Password_Value["state"] = "disabled"
        New_Entry_Button["state"] = "disabled"
        New_Entry_Update()
    else:
        Message_Box_Value["text"] = "Entered Wrong Admin Password...\nPlease Try Again!"

#Callback function when New Entry Button is pressed to create Popup
def New_Entry_Popup():
    New_Entry_Name_Label["state"] = "active"
    New_Entry_Name_Value["state"] = "normal"
    New_Entry_Password_Label["state"] = "active"
    New_Entry_Password_Value["state"] = "normal"
    New_Entry_Button["state"] = "active"
    Message_Box_Value["text"] = "Please Enter Required Details...\nRe-Training of Model will take a few minutes!\nThe GUI won't be responsive during this Period!"
            
#Callback function when Authenticate Button is pressed
def Authenticate():
    Liveliness_Detection()
    
Parent_Frame = FDAS.Frame(borderwidth=10, relief="groove")
Parent_Greeting = FDAS.Label(Parent_Frame, text="Welcome! \n Facial Detection & Authentication System\n", font="TkFixedFont")

IGControl_Frame = FDAS.Frame(Parent_Frame, borderwidth=3, relief="raised")
IGControl_Greeting = FDAS.Label(IGControl_Frame, text="Ignition Control\n", font="TkIconFont")
IGButton_On = FDAS.Button(IGControl_Frame, text='ON', command=IgnitionOn)
IGButton_Off = FDAS.Button(IGControl_Frame, text='OFF', command=IgnitionOff)
IGButton_Off["state"] = "disabled"

Speed_Frame = FDAS.Frame(Parent_Frame, borderwidth=3, relief="sunken")
Speed_Greeting = FDAS.Label(Speed_Frame, text="Speed\n", font="TkIconFont")
Speed_Value = FDAS.Label(Speed_Frame, text=speed, font="TkTextFont")
Speed_Unit = FDAS.Label(Speed_Frame, text="Kmph", font="TkCaptionFont")
Speed_Value["state"] = "disabled"
Speed_Unit["state"] = "disabled"

FDAS_Control_Frame = FDAS.Frame(Parent_Frame, borderwidth=3, relief="raised")
FDAS_Control_Greeting = FDAS.Label(FDAS_Control_Frame, text="FDAS Control\n", font="TkIconFont")
FDAS_Button_Auth = FDAS.Button(FDAS_Control_Frame, text='Authenticate', command=Authenticate)
FDAS_Button_New = FDAS.Button(FDAS_Control_Frame, text='New Entry', command=New_Entry_Popup)
FDAS_Button_Auth["state"] = "disabled"
FDAS_Button_New["state"] = "disabled"

Accelerator_Frame = FDAS.Frame(Parent_Frame, borderwidth=3, relief="raised")
Accelerator_Greeting = FDAS.Label(Accelerator_Frame, text="Accelerator\n", font="TkIconFont")
Accelerator_Value = FDAS.Scale(Accelerator_Frame, orient='horizontal', length=200, from_=0, to=100, variable = desired_speed,
                              command=Update_Accelerator_Label)
Accelerator_Value["state"] = "disabled"

Message_Box_Frame = FDAS.Frame(Parent_Frame, borderwidth=3, relief="sunken")
Message_Box_Greeting = FDAS.Label(Message_Box_Frame, text="Message Box\n", font="TkIconFont")
Message_Box_Value = FDAS.Label(Message_Box_Frame, text="System Messages will Display here...", font="TkSmallCaptionFont", 
                               width=35, height=5)
Message_Box_Value["state"] = "disabled"

New_Entry_Frame = FDAS.Frame(Parent_Frame, borderwidth=3, relief="raised")
New_Entry_Greeting = FDAS.Label(New_Entry_Frame, text="New Driver Entry\n", font="TkIconFont")
New_Entry_Name_Label = FDAS.Label(New_Entry_Frame, text="New Driver Name", font="TkSmallCaptionFont")
New_Driver_Name = FDAS.StringVar()

New_Entry_Name_Value = FDAS.Entry(New_Entry_Frame, textvariable=New_Driver_Name)
New_Entry_Password_Label = FDAS.Label(New_Entry_Frame, text="Admin Password", font="TkSmallCaptionFont")
Admin_Password = FDAS.StringVar()
New_Entry_Password_Value = FDAS.Entry(New_Entry_Frame, textvariable=Admin_Password, show='*')
New_Entry_Validate = partial(New_Entry_Validate, New_Driver_Name, Admin_Password)
New_Entry_Button = FDAS.Button(New_Entry_Frame, text='Validate', command=New_Entry_Validate)

New_Entry_Name_Label["state"] = "disabled"
New_Entry_Name_Value["state"] = "disabled"
New_Entry_Password_Label["state"] = "disabled"
New_Entry_Password_Value["state"] = "disabled"
New_Entry_Button["state"] = "disabled"

Parent_Frame.grid(column=0, row=0, padx=5, pady=5)
Parent_Greeting.grid(column=0, row=0, columnspan=3, padx=5, pady=5)
IGControl_Frame.grid(column=0, row=1, columnspan=1, rowspan=1, padx=5, pady=5)
IGControl_Greeting.grid(column=0, row=1, columnspan=1, padx=5, pady=5)
IGButton_On.grid(column=0, row=2, sticky = 'W', padx=5, pady=5)
IGButton_Off.grid(column=1, row=2, sticky = 'E', padx=5, pady=5)
Speed_Frame.grid(column=2, row=1, columnspan=1, rowspan=1, padx=5, pady=5)
Speed_Greeting.grid(column=2, row=1, columnspan=1, padx=5, pady=5)
Speed_Value.grid(column=2, row=2, padx=5, pady=5)
Speed_Unit.grid(column=3, row=2, padx=5, pady=5)
FDAS_Control_Frame.grid(column=0, row=3, columnspan=1, rowspan=1, padx=5, pady=5)
FDAS_Control_Greeting.grid(column=0, row=3, columnspan=1, padx=5, pady=5)
FDAS_Button_Auth.grid(column=0, row=4, sticky = 'W', padx=5, pady=5)
FDAS_Button_New.grid(column=1, row=4, sticky = 'E', padx=5, pady=5)
Accelerator_Frame.grid(column=2, row=3, columnspan=1, rowspan=1, padx=5, pady=5)
Accelerator_Greeting.grid(column=2, row=3, columnspan=1, padx=5, pady=5)
Accelerator_Value.grid(column=2, row=4, sticky = 'E', padx=5, pady=5)
Message_Box_Frame.grid(column=0, row=5, columnspan=2, rowspan=4, padx=5, pady=5)
Message_Box_Greeting.grid(column=0, row=5, columnspan=2, padx=5, pady=5)
Message_Box_Value.grid(column=0, row=6, columnspan=2, rowspan=3, padx=5, pady=5)
New_Entry_Frame.grid(column=2, row=5, columnspan=2, rowspan=4,padx=5, pady=5)
New_Entry_Greeting.grid(column=2, row=5, columnspan=2,padx=5, pady=5)
New_Entry_Name_Label.grid(column=2, row=6, padx=5, pady=5)
New_Entry_Name_Value.grid(column=3, row=6, padx=5, pady=5)
New_Entry_Password_Label.grid(column=2, row=7, padx=5, pady=5)
New_Entry_Password_Value.grid(column=3, row=7, padx=5, pady=5)
New_Entry_Button.grid(column=2, row=8, columnspan=2,padx=5, pady=5)

Parent_Frame.after(500, SimulateCar)
Parent_Frame.mainloop()

C:\Users\qv445\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
C:\Users\qv445\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\qv445\AppData\Local\Continuum\anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\qv445\AppData\Local\Continuum\anaconda3\lib\site-packages\requests\packages\urllib